In [78]:
import pandas as pd
import sqldf

In [79]:
sale = pd.read_csv('sale.csv')
## Convert to datetime on time columns
sale['date_acquired'] = pd.to_datetime(sale['date_acquired'])
sale['date_sold'] = pd.to_datetime(sale['date_sold'])
sale['days_on_market'] = (sale['date_sold'] - sale['date_acquired']).dt.days

sale.head()

,inventory_id,date_acquired,date_sold,type_of_property,buying_price,selling_price,days_on_market
0,CON000025,2021-05-18,2022-04-18,DEF,715500,698031.0,335.0
1,CON000041,2021-05-18,NaT,IBY,240345,NaN,NaN
2,CON000038,2021-05-20,2021-12-01,DEF,400000,360783.0,195.0
3,CON000031,2021-06-01,NaT,DEF,560870,NaN,NaN
4,CON000040,2021-06-05,2022-02-18,IBY,350456,438500.0,258.0


In [82]:
def calculate_sale_agg():
    sale_agg_1_query = '''
        SELECT s.inventory_id, s.type_of_property, s.date_acquired, s.date_sold, s.days_on_market, COALESCE(i.cost, 0) AS cost, s.selling_price - (s.buying_price + COALESCE(i.cost, 0)) AS profit
        FROM sale AS s
        LEFT JOIN inventory_id_cost AS i ON s.inventory_id = i.inventory_id
    '''
    sale_agg_1 = sqldf.run(sale_agg_1_query)

    sale_agg_2_query = '''
    SELECT date_sold, type_of_property, AVG(days_on_market) AS avg_days_on_market, SUM(profit) AS profit
    FROM sale_agg_1
    WHERE date_sold IS NOT NULL
    GROUP BY date_sold, type_of_property
    ORDER BY date_sold, type_of_property
    '''
    # Thay thế 'sale_agg_1' bằng 'sale_agg_1_query' trong truy vấn 'sale_agg_2_query'
    sale_agg_2 = sqldf.run(sale_agg_2_query.replace('sale_agg_1', sale_agg_1_query))
    return sale_agg_2

In [54]:
mkt = pd.read_csv('mkt.csv')
mkt['date_of_disbursement'] = pd.to_datetime(mkt['date_of_disbursement'])

In [55]:
inventory_id_cost_query = '''
    SELECT inventory_id, SUM(cost) AS cost
    FROM mkt AS m
    GROUP BY inventory_id
'''
inventory_id_cost = sqldf.run(inventory_id_cost_query)

sale_agg_1_query = '''
    SELECT s.inventory_id, s.type_of_property, s.date_acquired, s.date_sold, s.days_on_market, COALESCE(i.cost, 0) AS cost, s.selling_price - (s.buying_price + COALESCE(i.cost, 0)) AS profit
    FROM sale AS s
    LEFT JOIN inventory_id_cost AS i ON s.inventory_id = i.inventory_id
'''
sale_agg_1 = sqldf.run(sale_agg_1_query)
sale_agg_1.head()

,inventory_id,type_of_property,date_acquired,date_sold,days_on_market,cost,profit
0,CON000025,DEF,2021-05-18 00:00:00,2022-04-18 00:00:00,335.0,89.80,-17558.80
1,CON000041,IBY,2021-05-18 00:00:00,None,NaN,38.65,NaN
2,CON000038,DEF,2021-05-20 00:00:00,2021-12-01 00:00:00,195.0,0.00,-39217.00
3,CON000031,DEF,2021-06-01 00:00:00,None,NaN,18.53,NaN
4,CON000040,IBY,2021-06-05 00:00:00,2022-02-18 00:00:00,258.0,12.94,88031.06


In [56]:
sale_agg_2_query = '''
    SELECT date_sold, type_of_property, AVG(days_on_market) AS avg_days_on_market, SUM(profit) AS profit
    FROM sale_agg
    WHERE date_sold IS NOT NULL
    GROUP BY date_sold, type_of_property
    ORDER BY date_sold, type_of_property
'''
sale_agg_2 = sqldf.run(sale_agg_2_query)
sale_agg_2.head()

,date_sold,type_of_property,avg_days_on_market,profit
0,2021-12-01 00:00:00,DEF,195.0,-39217.00
1,2022-02-14 00:00:00,DEF,48.0,29663.16
2,2022-02-18 00:00:00,IBY,258.0,88031.06
3,2022-03-21 00:00:00,IBY,71.0,10026.00
4,2022-03-27 00:00:00,IBY,89.0,135697.36


In [57]:
mkt_agg_query = '''
    SELECT date_of_disbursement, expendtiture_category, SUM(cost) AS cost
    FROM mkt AS m
    GROUP BY date_of_disbursement, expendtiture_category
    ORDER BY date_of_disbursement
'''
mkt_agg = sqldf.run(mkt_agg_query)
mkt_agg.head()

,date_of_disbursement,expendtiture_category,cost
0,2022-01-11 00:00:00,Boosting,28.29
1,2022-01-11 00:00:00,Listing,10.44
2,2022-01-16 00:00:00,Boosting,8.77
3,2022-01-16 00:00:00,Others,21.28
4,2022-01-18 00:00:00,Boosting,54.99


In [58]:
sale.head()

,inventory_id,date_acquired,date_sold,type_of_property,buying_price,selling_price,days_on_market
0,CON000025,2021-05-18 00:00:00,2022-04-18 00:00:00,DEF,715500,698031.0,335.0
1,CON000041,2021-05-18 00:00:00,None,IBY,240345,NaN,NaN
2,CON000038,2021-05-20 00:00:00,2021-12-01 00:00:00,DEF,400000,360783.0,195.0
3,CON000031,2021-06-01 00:00:00,None,DEF,560870,NaN,NaN
4,CON000040,2021-06-05 00:00:00,2022-02-18 00:00:00,IBY,350456,438500.0,258.0


In [60]:
inventory_id_cost_query = '''
    SELECT inventory_id, SUM(cost) AS cost
    FROM mkt AS m
    GROUP BY inventory_id
'''
inventory_id_cost = sqldf.run(inventory_id_cost_query)

In [76]:
# Top những bất động sản vẫn chưa bán được
query_1 = '''
    SELECT s.inventory_id, s.date_acquired, i.cost, CURRENT_DATE AS current_date
    FROM sale AS s
    LEFT JOIN inventory_id_cost AS i ON s.inventory_id = i.inventory_id
    WHERE s.date_sold IS NULL
    ORDER BY cost DESC
'''

df_1 = sqldf.run(query_1)
df_1.head()

,inventory_id,cost,current_date
0,CON000016,220.31,2024-02-24
1,CON000056,135.38,2024-02-24
2,CON000060,94.90,2024-02-24
3,CON000035,62.84,2024-02-24
4,CON000059,60.84,2024-02-24
